In [ ]:
import sys
import os
import time
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
import psutil
import GPUtil


import torch
import torch.nn as nn
import zarr
import xarray as xr
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.spatial.distance import cosine


sys.path.append('..')


from data_build.metadata_db import MetadataManager, DataDomain, StorageTier
from surrogate import get_surrogate_manager, SurrogateMode
from datamodules.cube_dm import CubeDM
from pipeline.pipeline_run import PipelineRunner
from validation.eval_cube import CubeEvaluator
from models.datacube_unet import CubeUNet
from models.surrogate_transformer import SurrogateTransformer
from models.graph_vae import GVAE
from utils.config import load_config

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("🚀 Comprehensive Scientific Data Validation Suite")
print("=" * 60)
print(f"Started at: {datetime.now()}")
print(f"System: {os.uname().sysname} {os.uname().release}")
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("=" * 60)


In [ ]:
class ValidationConfig:
    """Configuration for validation suite"""

    def __init__(self):
        self.max_runtime_minutes = 30  # Maximum runtime
        self.sample_size_limit = 1000  # Samples per domain for testing
        self.batch_size = 16
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Performance thresholds
        self.min_r2_score = 0.95
        self.max_latency_ms = 400
        self.max_energy_error = 0.01  # 1%
        self.max_mass_error = 0.005   # 0.5%

        # Data domains to validate
        self.domains = [
            DataDomain.ASTRONOMICAL,
            DataDomain.EXOPLANET,
            DataDomain.ENVIRONMENTAL,
            DataDomain.PHYSICS,
            DataDomain.OPTICAL,
            DataDomain.PHYSIOLOGICAL,
            DataDomain.BIOSIGNATURE
        ]

        # Model types to test
        self.model_types = [
            "CubeUNet",
            "SurrogateTransformer",
            "GVAE",
            "FusionModel"
        ]

config = ValidationConfig()
print(f"✅ Validation configuration loaded")
print(f"   Max runtime: {config.max_runtime_minutes} minutes")
print(f"   Sample limit: {config.sample_size_limit} per domain")
print(f"   Device: {config.device}")
print(f"   Domains: {len(config.domains)}")
print(f"   Model types: {len(config.model_types)}")


In [ ]:
# Initialize metadata manager
metadata_manager = MetadataManager()

# Get database statistics
stats = metadata_manager.get_dataset_statistics()
print(f"📊 Database Statistics:")
print(f"   Total datasets: {stats['total_datasets']}")
print(f"   Total experiments: {stats['total_experiments']}")
print(f"   Total data chunks: {stats['total_data_chunks']}")
print(f"   Total size: {stats['total_size_tb']:.2f} TB")

# Check data availability for each domain
domain_data = {}
for domain in config.domains:
    datasets = metadata_manager.query_datasets(domain=domain.value)
    domain_data[domain] = datasets
    print(f"   {domain.value}: {len(datasets)} datasets")

print(f"\n✅ Found data for {len([d for d in domain_data.values() if d])} domains")


In [ ]:
class ValidationRunner:
    """Main validation runner class"""

    def __init__(self, config: ValidationConfig):
        self.config = config
        self.results = {}
        self.start_time = time.time()

        # Initialize surrogate manager
        self.surrogate_manager = get_surrogate_manager()

        # Initialize evaluator
        self.evaluator = CubeEvaluator()

        # Performance monitoring
        self.performance_data = []

    def check_time_limit(self) -> bool:
        """Check if we're within time limit"""
        elapsed = (time.time() - self.start_time) / 60
        return elapsed < self.config.max_runtime_minutes

    def create_synthetic_data(self, domain: DataDomain, num_samples: int = 100) -> Dict[str, np.ndarray]:
        """Create synthetic data for domains without available datasets"""

        np.random.seed(42)  # Reproducible results

        if domain == DataDomain.ASTRONOMICAL:
            return {
                'inputs': np.random.uniform(0, 1, (num_samples, 64, 64, 32)),
                'targets': np.random.uniform(0, 1, (num_samples, 64, 64, 32)),
                'metadata': {'type': 'stellar_atmosphere', 'resolution': '64x64x32'}
            }
        elif domain == DataDomain.EXOPLANET:
            return {
                'inputs': np.random.uniform(0, 1, (num_samples, 128, 64, 16)),
                'targets': np.random.uniform(0, 1, (num_samples, 128, 64, 16)),
                'metadata': {'type': 'planetary_atmosphere', 'resolution': '128x64x16'}
            }
        elif domain == DataDomain.ENVIRONMENTAL:
            return {
                'inputs': np.random.uniform(0, 1, (num_samples, 256, 128, 64)),
                'targets': np.random.uniform(0, 1, (num_samples, 256, 128, 64)),
                'metadata': {'type': 'climate_datacube', 'resolution': '256x128x64'}
            }
        elif domain == DataDomain.PHYSICS:
            return {
                'inputs': np.random.uniform(0, 1, (num_samples, 32, 32, 32)),
                'targets': np.random.uniform(0, 1, (num_samples, 32, 32, 32)),
                'metadata': {'type': 'physics_simulation', 'resolution': '32x32x32'}
            }
        elif domain == DataDomain.OPTICAL:
            return {
                'inputs': np.random.uniform(0, 1, (num_samples, 2048)),
                'targets': np.random.uniform(0, 1, (num_samples, 2048)),
                'metadata': {'type': 'optical_spectra', 'resolution': '2048_channels'}
            }
        elif domain == DataDomain.PHYSIOLOGICAL:
            return {
                'inputs': np.random.uniform(0, 1, (num_samples, 500)),
                'targets': np.random.uniform(0, 1, (num_samples, 500)),
                'metadata': {'type': 'biosignature_vectors', 'features': 500}
            }
        else:
            # Generic data
            return {
                'inputs': np.random.uniform(0, 1, (num_samples, 100)),
                'targets': np.random.uniform(0, 1, (num_samples, 100)),
                'metadata': {'type': 'generic', 'features': 100}
            }

    def load_validation_data(self, domain: DataDomain) -> Dict[str, np.ndarray]:
        """Load validation data for a domain"""

        datasets = metadata_manager.query_datasets(domain=domain.value)

        if not datasets:
            print(f"   ⚠️  No datasets found for {domain.value}, creating synthetic data")
            return self.create_synthetic_data(domain, self.config.sample_size_limit)

        # Use first available dataset
        dataset = datasets[0]

        try:
            if dataset.zarr_path and Path(dataset.zarr_path).exists():
                # Load from zarr
                zarr_data = zarr.open(dataset.zarr_path, mode='r')

                # Sample data to fit within limits
                total_samples = zarr_data.shape[0] if hasattr(zarr_data, 'shape') else len(zarr_data)
                sample_indices = np.random.choice(
                    total_samples,
                    size=min(self.config.sample_size_limit, total_samples),
                    replace=False
                )

                if hasattr(zarr_data, 'inputs') and hasattr(zarr_data, 'targets'):
                    inputs = zarr_data.inputs[sample_indices]
                    targets = zarr_data.targets[sample_indices]
                else:
                    # Assume single array, split for inputs/targets
                    data = zarr_data[sample_indices]
                    inputs = data
                    targets = data  # Self-supervised for now

                return {
                    'inputs': inputs,
                    'targets': targets,
                    'metadata': dataset.dimensions or {}
                }
            else:
                print(f"   ⚠️  Zarr path not found for {dataset.name}, creating synthetic data")
                return self.create_synthetic_data(domain, self.config.sample_size_limit)

        except Exception as e:
            print(f"   ⚠️  Error loading {dataset.name}: {e}, creating synthetic data")
            return self.create_synthetic_data(domain, self.config.sample_size_limit)

# Initialize validation runner
validator = ValidationRunner(config)
print(f"✅ Validation runner initialized")


In [ ]:
# Add validation methods to ValidationRunner class
def validate_model_on_domain(self, model_type: str, domain: DataDomain) -> Dict[str, Any]:
    """Validate a model on a specific domain"""

    print(f"   🔍 Validating {model_type} on {domain.value}...")

    # Load validation data
    start_time = time.time()
    data = self.load_validation_data(domain)
    load_time = time.time() - start_time

    inputs = data['inputs']
    targets = data['targets']

    print(f"     📊 Data shape: {inputs.shape} -> {targets.shape}")
    print(f"     ⏱️  Load time: {load_time:.2f}s")

    # Get appropriate model
    try:
        if '4d' in model_type.lower() or 'cube' in model_type.lower():
            surrogate_mode = SurrogateMode.DATACUBE
        else:
            surrogate_mode = SurrogateMode.SCALAR

        model = self.surrogate_manager.get_model(surrogate_mode)

        if not model:
            print(f"     ⚠️  No model available for {surrogate_mode}")
            return {'status': 'no_model', 'error': f'No model for {surrogate_mode}'}

    except Exception as e:
        print(f"     ❌ Error loading model: {e}")
        return {'status': 'model_error', 'error': str(e)}

    # Run validation
    results = {'status': 'success', 'model_type': model_type, 'domain': domain.value}

    try:
        # Performance test
        latencies = []
        batch_size = min(self.config.batch_size, len(inputs))

        for i in range(0, len(inputs), batch_size):
            batch_inputs = inputs[i:i+batch_size]

            start_time = time.time()
            predictions = model.predict(batch_inputs)
            latency = time.time() - start_time

            latencies.append(latency * 1000 / len(batch_inputs))  # ms per sample

            if not self.check_time_limit():
                break

        # Calculate metrics
        avg_latency = np.mean(latencies)
        p95_latency = np.percentile(latencies, 95)

        results.update({
            'avg_latency_ms': avg_latency,
            'p95_latency_ms': p95_latency,
            'samples_processed': len(inputs),
            'load_time_s': load_time
        })

        # Accuracy test (sample a subset for speed)
        test_indices = np.random.choice(len(inputs), size=min(100, len(inputs)), replace=False)
        test_inputs = inputs[test_indices]
        test_targets = targets[test_indices]

        test_predictions = model.predict(test_inputs)

        # Convert to numpy if needed
        if hasattr(test_predictions, 'cpu'):
            test_predictions = test_predictions.cpu().numpy()
        if hasattr(test_targets, 'cpu'):
            test_targets = test_targets.cpu().numpy()

        # Calculate accuracy metrics
        pred_flat = test_predictions.flatten()
        target_flat = test_targets.flatten()

        r2 = r2_score(target_flat, pred_flat)
        rmse = np.sqrt(mean_squared_error(target_flat, pred_flat))
        mae = mean_absolute_error(target_flat, pred_flat)

        results.update({
            'r2_score': r2,
            'rmse': rmse,
            'mae': mae,
            'passes_r2_threshold': r2 >= self.config.min_r2_score,
            'passes_latency_threshold': p95_latency <= self.config.max_latency_ms
        })

        print(f"     ✅ R² = {r2:.4f}, RMSE = {rmse:.4f}, Latency = {p95_latency:.1f}ms")

    except Exception as e:
        print(f"     ❌ Validation error: {e}")
        results.update({'status': 'validation_error', 'error': str(e)})

    return results

def run_comprehensive_validation(self) -> Dict[str, Any]:
    """Run comprehensive validation across all domains and models"""

    print(f"\n🚀 Starting comprehensive validation...")
    print(f"   Max runtime: {self.config.max_runtime_minutes} minutes")

    all_results = {}

    for domain in self.config.domains:
        if not self.check_time_limit():
            print(f"   ⏰ Time limit reached, stopping validation")
            break

        print(f"\n📊 Validating domain: {domain.value}")
        domain_results = {}

        # Test available models for this domain
        for model_type in self.config.model_types:
            if not self.check_time_limit():
                break

            result = self.validate_model_on_domain(model_type, domain)
            domain_results[model_type] = result

        all_results[domain.value] = domain_results

    # Summary statistics
    summary = self.generate_summary(all_results)

    return {
        'results': all_results,
        'summary': summary,
        'performance_data': self.performance_data,
        'validation_time': time.time() - self.start_time
    }

# Add methods to ValidationRunner
ValidationRunner.validate_model_on_domain = validate_model_on_domain
ValidationRunner.run_comprehensive_validation = run_comprehensive_validation

print("✅ Validation methods added to ValidationRunner")


In [ ]:
# Add summary generation method
def generate_summary(self, results: Dict[str, Any]) -> Dict[str, Any]:
    """Generate summary statistics"""

    summary = {
        'total_validations': 0,
        'successful_validations': 0,
        'avg_r2_score': 0,
        'avg_latency_ms': 0,
        'domains_tested': len(results),
        'models_tested': len(self.config.model_types),
        'passes_all_thresholds': True
    }

    r2_scores = []
    latencies = []

    for domain_name, domain_results in results.items():
        for model_type, result in domain_results.items():
            summary['total_validations'] += 1

            if result.get('status') == 'success':
                summary['successful_validations'] += 1

                if 'r2_score' in result:
                    r2_scores.append(result['r2_score'])
                if 'p95_latency_ms' in result:
                    latencies.append(result['p95_latency_ms'])

                # Check thresholds
                if not result.get('passes_r2_threshold', True):
                    summary['passes_all_thresholds'] = False
                if not result.get('passes_latency_threshold', True):
                    summary['passes_all_thresholds'] = False

    if r2_scores:
        summary['avg_r2_score'] = np.mean(r2_scores)
        summary['min_r2_score'] = np.min(r2_scores)
        summary['max_r2_score'] = np.max(r2_scores)

    if latencies:
        summary['avg_latency_ms'] = np.mean(latencies)
        summary['max_latency_ms'] = np.max(latencies)
        summary['min_latency_ms'] = np.min(latencies)

    summary['success_rate'] = summary['successful_validations'] / max(1, summary['total_validations'])

    return summary

# Add method to ValidationRunner
ValidationRunner.generate_summary = generate_summary

print("✅ Summary generation method added")


In [ ]:
# Run the comprehensive validation
print(f"\n🚀 STARTING COMPREHENSIVE VALIDATION")
print(f"="*60)

validation_results = validator.run_comprehensive_validation()

print(f"\n✅ VALIDATION COMPLETE")
print(f"="*60)
print(f"Total time: {validation_results['validation_time']:.2f} seconds")
print(f"Total validations: {validation_results['summary']['total_validations']}")
print(f"Successful validations: {validation_results['summary']['successful_validations']}")
print(f"Success rate: {validation_results['summary']['success_rate']:.2%}")
print(f"Average R² score: {validation_results['summary']['avg_r2_score']:.4f}")
print(f"Average latency: {validation_results['summary']['avg_latency_ms']:.1f}ms")
print(f"Passes all thresholds: {validation_results['summary']['passes_all_thresholds']}")

# Create detailed results table
results_data = []

for domain_name, domain_results in validation_results['results'].items():
    for model_type, result in domain_results.items():
        results_data.append({
            'Domain': domain_name,
            'Model': model_type,
            'Status': result.get('status', 'unknown'),
            'R² Score': result.get('r2_score', 0),
            'RMSE': result.get('rmse', 0),
            'MAE': result.get('mae', 0),
            'P95 Latency (ms)': result.get('p95_latency_ms', 0),
            'Samples': result.get('samples_processed', 0),
            'Load Time (s)': result.get('load_time_s', 0),
            'Passes R² Threshold': result.get('passes_r2_threshold', False),
            'Passes Latency Threshold': result.get('passes_latency_threshold', False)
        })

results_df = pd.DataFrame(results_data)

# Display results table
print("\n📊 DETAILED VALIDATION RESULTS")
print("="*80)
print(results_df.to_string(index=False))
